In [1]:
import numpy as np
from sklearn.neighbors import KDTree
import map_elites.cvt as cvt_map_elites
import map_elites.common as cm_map_elites
import cairo
from PIL import Image

In [3]:
def load_data(filename, dim,dim_x):
    print("Loading ",filename)
    data = np.loadtxt(filename)
    fit = data[:, 0:1]
    cent = data[:,1: dim+1]
    desc = data[:,dim+1: 2*dim+1]
    x = data[:,2*dim+1:2*dim+1+dim_x]
    
def load_centroids(filename):
    points = np.loadtxt(filename)
    return points

def make_hashable(array):
    return tuple(map(float, array))

def getKDT(n_niches, dim_map, number):

    fname = "centroids_{}.dat".format(number)
    c = np.loadtxt(fname)
    kdt = KDTree(c, leaf_size=30, metric='euclidean')
    return kdt

def draw(geno):
    surface = cairo.ImageSurface(cairo.FORMAT_ARGB32, 10, 10)
    ctx = cairo.Context(surface)
    ctx.set_source_rgba(geno[0], geno[1], geno[2], 1)
    ctx.paint()
    return surface

def pilImageFromCairoSurface( surface ):
    cairoFormat = surface.get_format()
    if cairoFormat == cairo.FORMAT_ARGB32:
        pilMode = 'RGB'
        # Cairo has ARGB. Convert this to RGB for PIL which supports only RGB or
        # RGBA.
        argbArray = np.frombuffer( surface.get_data().tobytes(), 'c' ).reshape( -1, 4 )
        rgbArray = argbArray[ :, 2::-1 ]
        pilData = rgbArray.reshape( -1 ).tobytes()
    else:
        raise ValueError( 'Unsupported cairo format: %d' % cairoFormat )
    pilImage = Image.frombuffer( pilMode,
    ( surface.get_width(), surface.get_height() ), pilData, "raw", pilMode, 0, 1 )
    pilImage = pilImage.convert( 'RGB' )
    return pilImage

if __name__ == "__main__":

    spr_geno = []
    spr_pheno = []
    for k in range(1,10):
        
        centroids = load_centroids("centroids_{}.dat".format(k))
        data = np.loadtxt("archive_{}.dat".format(k))
        fit = data[:,0:1]
        cent = data[:, 1:3]
        desc = data[:, 3: 5]
        geno = data[:, 5: 9]
        #print("Fit: {}".format(fit[0]))
        #print("Cent: {}".format(cent[0]))
        #print("Behavior: {}".format(desc[0]))
        #print("Geno: {}".format(geno[0]))

        test1 = np.array([0, 0, 0])
        test2 = np.array([1, 1, 1])

        max_pos = np.linalg.norm(test1- test2)
        #print("Maximum possible distance: {}".format(max_pos))

        #print("-------------")

        archive = {}
        for j in range(0, len(fit)):
            archive[tuple(cent[j])] = [geno[j]]

        kdt = getKDT(1000, 2, k)
        distances_geno = []
        distances_pheno = []
        for i in range(0, len(fit)):
            try:
                test = kdt.query([np.array([desc[i][0], desc[i][1]])], k=2)[1][0]
                niche_1= kdt.data[test[0]]
                niche_2= kdt.data[test[1]]
                n1 = make_hashable(niche_1)
                n2 = make_hashable(niche_2)
                uno = np.array(archive[n1][0])
                due = np.array(archive[n2][0])

                dist_geno = np.linalg.norm(uno - due)
                distances_geno.append(dist_geno/max_pos)

                img_uno = draw(uno)
                img_due = draw(due)
                #imgP1 = Image.frombuffer("RGBA",(img_uno.get_width(),img_uno.get_height() ),img_uno.get_data(),"raw","RGBA",0,1)
                #imgP2 = Image.frombuffer("RGBA",(img_due.get_width(),img_due.get_height() ),img_due.get_data(),"raw","RGBA",0,1)
                imgP1 = pilImageFromCairoSurface(img_uno)
                imgP2 = pilImageFromCairoSurface(img_due)

                img_HSV_uno = imgP1.convert('HSV')
                img_HSV_due = imgP2.convert('HSV')
                img_HSV_arr_uno = np.array(img_HSV_uno)
                img_HSV_arr_due = np.array(img_HSV_due)
                check = [img_HSV_arr_uno[5,5,0], img_HSV_arr_uno[5,5,1], img_HSV_arr_uno[5,5,2]]

                one = [img_HSV_arr_uno[5,5,0]/255, img_HSV_arr_uno[5,5,1]/255, img_HSV_arr_uno[5,5,2]/255]
                two = [img_HSV_arr_due[5,5,0]/255, img_HSV_arr_due[5,5,1]/255, img_HSV_arr_due[5,5,2]/255]
                one = np.array(one)
                #print(one)
                #print('-----')
                two = np.array(two)
                dist_pheno = np.linalg.norm(one - two)
                distances_pheno.append(dist_pheno/max_pos)

            except:
                pass

        spr_geno.append(sum(distances_geno)/len(distances_geno))
        spr_pheno.append(sum(distances_pheno)/len(distances_pheno))
        
    print("Finished")
    spread_geno = sum(spr_geno) / len(spr_geno)    
    spread_pheno = sum(spr_pheno) / len(spr_pheno)   
    print("Genotype Spread : {}".format(spread_geno))
    print("Phenotype Spread : {}".format(spread_pheno))
        

Finished
Genotype Spread : 0.03117574889577151
Phenotype Spread : 0.05028297316787507
